In [1]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../Examples/config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-06-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  parser.get('openai_api','api_model')

In [2]:
prompt= '''Provide conversation summary points of the following text:

Text:
###
{'displayName': 'Conversation Task Example', 'analysisInput': {'conversations': [{'conversationItems': [{'text': 'Hi, thank you for calling Contoso pharmacy. Who am I speaking with today?', 'id': 1, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Good afternoon. My name is Mary. I'm calling about a refill for my prescribed medications. I have been trying to get ahold of someone for weeks and was told that I would get a call back regarding my situation, but it's been weeks and no one's contacted me, so I thought I'd call.", 'id': 2, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': "I understand your frustration, Mary. Umm, can you tell me what exactly you're trying to accomplish?", 'id': 3, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Yes, I'm trying to get a refill of my prescription drugs that the my doctor prescribed to me for cholesterol.", 'id': 4, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': 'I see here that you were on a generic form of Lipitor, is that right?', 'id': 5, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': 'Uh, yes, I was taking the generic form of Lipitor.', 'id': 6, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': 'OK, uh, so I see that your doctor stopped prescribing these drugs in 2021, actually.', 'id': 7, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Oh really? That doesn't sound right. I don't remember him cancelling my prescription.", 'id': 8, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': "OK, uh, yeah, I'd be happy to check that for you. Uh, because sometimes there's a gap in the system and it just doesn't get reported. So let me take a look here.", 'id': 9, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': 'Just a moment.', 'id': 10, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': "So I'm seeing here that your doctor had these drugs prescribed to you from 2012 through 2021.", 'id': 11, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Ohh huh, I mean, I'm definitely supposed to be taking something else. Uh, would you check please?", 'id': 12, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': "OK. Uh, yeah. According to the latest records, uh, provided by doctor's office, you're now on a different drug, metformin. Would you like us to go ahead and fill that prescription for you for pick up a bit of form in 500 milligrams?", 'id': 13, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Yeah, yeah. Thank you so much. I'm almost out, so that'd be perfect.", 'id': 14, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': "Uh, you're very welcome, Mary. Please let us know if there's anything else we can do for you today.", 'id': 15, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': 'OK. Thank you.', 'id': 16, 'role': 'Customer', 'participantId': 'Customer_2'}], 'modality': 'text', 'id': 'conversation1', 'language': 'en'}]}, 'tasks': [{'taskName': 'analyze 1', 'kind': 'ConversationalSummarizationTask', 'parameters': {'summaryAspects': ['chaptertitle', 'issue', 'resolution', 'narrative']}}]}
###

Result:
-'''

In [3]:
response = openai.Completion.create( engine='davinci2',  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
a = '-'+response['choices'][0]['text']
print(a)

Response:
- The customer is trying to refill a prescription for cholesterol medication, but the doctor stopped prescribing the medication in 2021.
- The agent checks the records and finds that the customer is now on a different medication, metformin.
- The agent offers to fill the prescription for the customer.
- The customer thanks the agent and ends the call.


In [4]:
response = openai.Completion.create( engine='text-curie-001',  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
b = '-'+response['choices'][0]['text']
print(b)

Response:
-The caller, Mary, is trying to get a refill for her prescribed medications that the her doctor prescribed to her for cholesterol.
-According to the latest records, she is now on a different drug, metformin.
-The agent can check to see if there is a gap in the system that may have prevented her from being contacted about her prescription.


In [5]:
import fkscore

f = fkscore.fkscore(a)
print(f.stats)
print(f.score)

{'num_sentences': 4, 'num_words': 60, 'num_syllables': 94}
{'readability': 59.07, 'grade': '11th Grade'}


In [6]:
import fkscore

f = fkscore.fkscore(b)
print(f.stats)
print(f.score)

{'num_sentences': 3, 'num_words': 60, 'num_syllables': 91}
{'readability': 58.225, 'grade': '11th Grade'}


In [7]:
print([a])

['- The customer is trying to refill a prescription for cholesterol medication, but the doctor stopped prescribing the medication in 2021.\n- The agent checks the records and finds that the customer is now on a different medication, metformin.\n- The agent offers to fill the prescription for the customer.\n- The customer thanks the agent and ends the call.']


In [8]:
a_notes = (a.replace('-','').split('\n'))

In [9]:
b_notes = (b.replace('-','').split('\n'))

In [10]:
sen = a_notes

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#Encoding:
sen_embeddings = model.encode(sen)
sen_embeddings.shape

(4, 384)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
#let's calculate cosine similarity for sentence 0:
cosine_similarity(
    [sen_embeddings[0]],
    sen_embeddings[1:]
)

array([[0.40374172, 0.4934861 , 0.25284168]], dtype=float32)

In [12]:
a_notes

[' The customer is trying to refill a prescription for cholesterol medication, but the doctor stopped prescribing the medication in 2021.',
 ' The agent checks the records and finds that the customer is now on a different medication, metformin.',
 ' The agent offers to fill the prescription for the customer.',
 ' The customer thanks the agent and ends the call.']

In [13]:
a_embeddings = model.encode(a_notes)
a_embeddings.shape

b_embeddings = model.encode(b_notes)
b_embeddings.shape

(3, 384)

In [56]:
similarity_matrix = cosine_similarity(
    a_embeddings, b_embeddings
)
print(type(similarity_matrix))

<class 'numpy.ndarray'>


In [57]:
a_notes

[' The customer is trying to refill a prescription for cholesterol medication, but the doctor stopped prescribing the medication in 2021.',
 ' The agent checks the records and finds that the customer is now on a different medication, metformin.',
 ' The agent offers to fill the prescription for the customer.',
 ' The customer thanks the agent and ends the call.']

In [58]:
b_notes

['The caller, Mary, is trying to get a refill for her prescribed medications that the her doctor prescribed to her for cholesterol.',
 'According to the latest records, she is now on a different drug, metformin.',
 'The agent can check to see if there is a gap in the system that may have prevented her from being contacted about her prescription.']

In [59]:
import numpy as np
similarity_matrix

array([[ 0.68625665,  0.29286778,  0.46407583],
       [ 0.42803872,  0.54409283,  0.6305856 ],
       [ 0.50612473,  0.20715195,  0.64771795],
       [ 0.341794  , -0.00269258,  0.41527697]], dtype=float32)

In [60]:
np.amax(similarity_matrix, axis=1)

array([0.68625665, 0.6305856 , 0.64771795, 0.41527697], dtype=float32)

In [61]:
np.mean(np.amax(similarity_matrix, axis=1))

0.5949593

In [51]:
np.std(np.amax(sim_mat, axis=1))

0.10568072

In [52]:
np.var(np.amax(sim_mat, axis=1))

0.011168414

In [50]:
np.mean(np.amax(sim_mat, axis=0))

0.6260225

In [27]:
np.var(np.amax(sim_mat, axis=1))

0.011168414

In [28]:
np.var(np.amax(sim_mat, axis=0))

0.003603772

In [29]:
a_notes_sim

0.5949593

In [54]:
np.count_nonzero(np.amax(sim_mat, axis=1) < .5)/ len(a_notes)

0.25

In [55]:
np.count_nonzero(np.amax(sim_mat, axis=0) < .5)/ len(b_notes)

0.0

In [43]:
np.count_nonzero(np.amax(sim_mat, axis=0) <  np.mean(np.amax(sim_mat, axis=0))-np.std(np.amax(sim_mat, axis=0)))

1